CIND 820- Big Data Analytics Project

Using Machine Learning for Prediction of Early Readmission of Diabetic Patients

Supervised by: Ceni Babaoglu

Presented by: Nehal Gamal Mohamed (501278190)


In [ ]:
!pip install pandas
import sys
!pip install matplotlib
!pip install graphviz

In [1]:
#importing necessary libraries
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [2]:
#Uploading diabetic_data csv file
from google.colab import files
data = files.upload()
#Instintiating DataReader class
class DataReader:
    def read_csv(self, filename):
        df = pd.read_csv(filename)
        return df

filename = next(iter(data))
data_reader = DataReader()
df = data_reader.read_csv(filename)
#Displaying the first 10 records of the dataset
print(df.head(10))


Saving diabetic_data (version 1).csv to diabetic_data (version 1).csv
   encounter_id  patient_nbr             race  gender       age weight  \
0       2278392      8222157        Caucasian  Female    [0-10)      ?   
1        149190     55629189        Caucasian  Female   [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female   [20-30)      ?   
3        500364     82442376        Caucasian    Male   [30-40)      ?   
4         16680     42519267        Caucasian    Male   [40-50)      ?   
5         35754     82637451        Caucasian    Male   [50-60)      ?   
6         55842     84259809        Caucasian    Male   [60-70)      ?   
7         63768    114882984        Caucasian    Male   [70-80)      ?   
8         12522     48330783        Caucasian  Female   [80-90)      ?   
9         15738     63555939        Caucasian  Female  [90-100)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        

In [3]:
#Displaying variables data types
print("Dataset Information:")
print(df.info())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-n

In [4]:
#Defining lists for categorical variables and numeric variables
categorical_columns = [
    'race',
    'gender',
    'age',
    'weight',
    'payer_code',
    'medical_specialty',
    'diag_1',
    'diag_2',
    'diag_3',
    'max_glu_serum',
    'A1Cresult',
    'metformin',
    'repaglinide',
    'nateglinide',
    'chlorpropamide',
    'glimepiride',
    'acetohexamide',
    'glipizide',
    'glyburide',
    'tolbutamide',
    'pioglitazone',
    'rosiglitazone',
    'acarbose',
    'miglitol',
    'troglitazone',
    'tolazamide',
    'examide',
    'citoglipton',
    'insulin',
    'glyburide-metformin',
    'glipizide-metformin',
    'glimepiride-pioglitazone',
    'metformin-rosiglitazone',
    'metformin-pioglitazone',
    'change',
    'diabetesMed',
    'readmitted'
]

numeric_columns = [
    'encounter_id',
    'patient_nbr',
    'admission_type_id',
    'discharge_disposition_id',
    'admission_source_id',
    'time_in_hospital',
    'num_lab_procedures',
    'num_procedures',
    'num_medications',
    'number_outpatient',
    'number_emergency',
    'number_inpatient',
    'number_diagnoses'
]


In [5]:
#Dataset Description
print("Dataset Description:")
print(df.describe(include='all'))

Dataset Description:
        encounter_id   patient_nbr       race  gender      age  weight  \
count   1.017660e+05  1.017660e+05     101766  101766   101766  101766   
unique           NaN           NaN          6       3       10      10   
top              NaN           NaN  Caucasian  Female  [70-80)       ?   
freq             NaN           NaN      76099   54708    26068   98569   
mean    1.652016e+08  5.433040e+07        NaN     NaN      NaN     NaN   
std     1.026403e+08  3.869636e+07        NaN     NaN      NaN     NaN   
min     1.252200e+04  1.350000e+02        NaN     NaN      NaN     NaN   
25%     8.496119e+07  2.341322e+07        NaN     NaN      NaN     NaN   
50%     1.523890e+08  4.550514e+07        NaN     NaN      NaN     NaN   
75%     2.302709e+08  8.754595e+07        NaN     NaN      NaN     NaN   
max     4.438672e+08  1.895026e+08        NaN     NaN      NaN     NaN   

        admission_type_id  discharge_disposition_id  admission_source_id  \
count       10

In [6]:
#Converted blank and "?" to NaN to represent missing values
# Replace '?' and blank cells with NaN
df.replace({'?': pd.NA, '': pd.NA}, inplace=True)

# Replaced 'None' in 'max_glu_serum' and 'A1Cresult' to be seen as 'No Test' category and not as missing values
df['max_glu_serum'].replace({pd.NA: 'No Test', 'None': 'No Test'}, inplace=True)
df['A1Cresult'].replace({pd.NA: 'No Test', 'None': 'No Test'}, inplace=True)
missing_values = df.isna().sum()
print("Number of missing values in each column:")
print(missing_values)

Number of missing values in each column:
encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepi

In [7]:
# converting numeric columns to numeric data types and categorical columns to category data type
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype('category')
print(df.dtypes)
print(df[numeric_columns].dtypes)


encounter_id                   int64
patient_nbr                    int64
race                        category
gender                      category
age                         category
weight                      category
admission_type_id              int64
discharge_disposition_id       int64
admission_source_id            int64
time_in_hospital               int64
payer_code                  category
medical_specialty           category
num_lab_procedures             int64
num_procedures                 int64
num_medications                int64
number_outpatient              int64
number_emergency               int64
number_inpatient               int64
diag_1                      category
diag_2                      category
diag_3                      category
number_diagnoses               int64
max_glu_serum               category
A1Cresult                   category
metformin                   category
repaglinide                 category
nateglinide                 category
c

In [8]:
# Handle missing values by replacing with mode
for column in ['race', 'diag_1', 'diag_2', 'diag_3']:
    df[column].fillna(df[column].mode()[0], inplace=True)

# Verify changes
print(df.isnull().sum())


encounter_id                    0
patient_nbr                     0
race                            0
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [9]:
# Drop unnecessary columns
df.drop(columns=['weight', 'payer_code', 'medical_specialty', 'encounter_id', 'patient_nbr'], inplace=True)

# Verify the remaining columns
print("Remaining columns after dropping unnecessary ones:")
print(df.columns)


Remaining columns after dropping unnecessary ones:
Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [10]:
#Checking for duplicate records
duplicate_records = df.duplicated().sum()
print(f"Number of duplicate records: {duplicate_records}")

Number of duplicate records: 0


In [11]:
# Simplify and categorize diag_1, diag_2, diag_3
def simplify_diag(x):
    if pd.isna(x):
        return -1
    if 'V' in x or 'E' in x:
        return 0
    try:
        return int(x[:3])
    except:
        return -1

df['diag_1'] = df['diag_1'].apply(simplify_diag)
df['diag_2'] = df['diag_2'].apply(simplify_diag)
df['diag_3'] = df['diag_3'].apply(simplify_diag)

# Verify changes
print(df[['diag_1', 'diag_2', 'diag_3']].head())


   diag_1  diag_2  diag_3
0     250     276     250
1     276     250     255
2     648     250       0
3       8     250     403
4     197     157     250


In [12]:
# Combine 'NO' and '>30' categories into a single '>30' category
df['readmitted'] = df['readmitted'].replace({'NO': '>30', '>30': '>30', '<30': '<30'})

# Verify changes
print(df['readmitted'].value_counts())


readmitted
>30    90409
<30    11357
Name: count, dtype: int64


In [13]:
# Here I dropped the List of columns dominated by "No" category since they have very low variance and no added value
columns_dominated_by_no = [
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
    'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
    'troglitazone', 'tolazamide', 'examide', 'citoglipton',
    'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
    'metformin-rosiglitazone', 'metformin-pioglitazone'
]
df.drop(columns=columns_dominated_by_no, inplace=True)

# Check the remaining columns
print("Remaining columns after dropping unnecessary ones:")
print(df.columns)
print(df.info())

Remaining columns after dropping unnecessary ones:
Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'miglitol', 'insulin', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   race                      101766 non-null  category
 1   gender                    101766 non-null  category
 2   age                       101766 non-null  category
 3   admission_type_id         101766 non-null  int64   
 4   discharge_disposition_id  101766 non-null  int6

In [14]:
# Ensure that 'admission_type_id', 'discharge_disposition_id', 'admission_source_id' are treated as categorical
df[['admission_type_id', 'discharge_disposition_id', 'admission_source_id']] = df[['admission_type_id', 'discharge_disposition_id', 'admission_source_id']].astype(str)

# List of categorical features
categorical_features = ['race', 'gender', 'age', 'max_glu_serum', 'A1Cresult', 'miglitol', 'insulin', 'change', 'diabetesMed',
                        'admission_type_id', 'discharge_disposition_id', 'admission_source_id']

# Convert all categorical columns to strings
df[categorical_features] = df[categorical_features].astype(str)

# Verify conversion
print(df[categorical_features].dtypes)


race                        object
gender                      object
age                         object
max_glu_serum               object
A1Cresult                   object
miglitol                    object
insulin                     object
change                      object
diabetesMed                 object
admission_type_id           object
discharge_disposition_id    object
admission_source_id         object
dtype: object


In [15]:
# One-hot encode categorical features
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Verify the changes
print(df.head())
print(df.info())


   time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0                 1                  41               0                1   
1                 3                  59               0               18   
2                 2                  11               5               13   
3                 2                  44               1               16   
4                 1                  51               0                8   

   number_outpatient  number_emergency  number_inpatient  diag_1  diag_2  \
0                  0                 0                 0     250     276   
1                  0                 0                 0     276     250   
2                  2                 0                 1     648     250   
3                  0                 0                 0       8     250   
4                  0                 0                 0     197     157   

   diag_3  ...  admission_source_id_20 admission_source_id_22  \
0     250  ...       

In [16]:
# Convert boolean columns to integer
bool_columns = df.select_dtypes(include='bool').columns
df[bool_columns] = df[bool_columns].astype(int)

# Verify the changes
print(df.head())
print(df.info())


   time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0                 1                  41               0                1   
1                 3                  59               0               18   
2                 2                  11               5               13   
3                 2                  44               1               16   
4                 1                  51               0                8   

   number_outpatient  number_emergency  number_inpatient  diag_1  diag_2  \
0                  0                 0                 0     250     276   
1                  0                 0                 0     276     250   
2                  2                 0                 1     648     250   
3                  0                 0                 0       8     250   
4                  0                 0                 0     197     157   

   diag_3  ...  admission_source_id_20 admission_source_id_22  \
0     250  ...       

In [17]:
# Convert the target variable 'readmitted' into binary (1 for '<30', 0 for '>30')
df['readmitted'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)


In [18]:
# Separate the features (X) and the labels (y)
X = df.drop(columns=['readmitted'])
y = df['readmitted']



In [19]:
from sklearn.model_selection import train_test_split
# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verify the split
print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)


Training set size: (71236, 88)
Test set size: (30530, 88)


In [20]:
from sklearn.preprocessing import StandardScaler

# Normalization stage
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [21]:
from imblearn.combine import SMOTETomek

# Resample the training dataset
smote_tomek = SMOTETomek(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train_scaled, y_train)

print("Original class distribution:", y_train.value_counts())
print("Resampled class distribution:", pd.Series(y_resampled).value_counts())



Original class distribution: readmitted
0    63304
1     7932
Name: count, dtype: int64
Resampled class distribution: readmitted
1    63230
0    63230
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Initialize models
log_reg = LogisticRegression(max_iter=5000, solver='saga', class_weight='balanced')
decision_tree = DecisionTreeClassifier(class_weight='balanced')
random_forest = RandomForestClassifier(class_weight='balanced')
gradient_boosting = GradientBoostingClassifier()

# Fit models on resampled training data
log_reg.fit(X_resampled, y_resampled)
decision_tree.fit(X_resampled, y_resampled)
random_forest.fit(X_resampled, y_resampled)
gradient_boosting.fit(X_resampled, y_resampled)



GradientBoostingClassifier()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

models = {
    "Logistic Regression": log_reg,
    "Decision Tree": decision_tree,
    "Random Forest": random_forest,
    "Gradient Boosting": gradient_boosting
}

for model_name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    print(f"\n{model_name} on Test Data:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, pos_label=1))
    print("Recall:", recall_score(y_test, y_pred, pos_label=1))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))



Logistic Regression on Test Data:
Accuracy: 0.6512938093678349
Precision: 0.1737008585630366
Recall: 0.5611678832116789
Confusion Matrix:
 [[17962  9143]
 [ 1503  1922]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.66      0.77     27105
           1       0.17      0.56      0.27      3425

    accuracy                           0.65     30530
   macro avg       0.55      0.61      0.52     30530
weighted avg       0.84      0.65      0.71     30530


Decision Tree on Test Data:
Accuracy: 0.7933180478218146
Precision: 0.15232586165341047
Recall: 0.18452554744525548
Confusion Matrix:
 [[23588  3517]
 [ 2793   632]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.87      0.88     27105
           1       0.15      0.18      0.17      3425

    accuracy                           0.79     30530
   macro avg       0.52      0.53      0.52     30530
weighted avg   

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Perform 10-fold cross-validation predictions for each model
y_pred_log_reg = cross_val_predict(log_reg, X_resampled, y_resampled, cv=10)
y_pred_decision_tree = cross_val_predict(decision_tree, X_resampled, y_resampled, cv=10)
y_pred_random_forest = cross_val_predict(random_forest, X_resampled, y_resampled, cv=10)
y_pred_gradient_boosting = cross_val_predict(gradient_boosting, X_resampled, y_resampled, cv=10)

# Calculate and print metrics for Logistic Regression
log_reg_accuracy = accuracy_score(y_resampled, y_pred_log_reg)
log_reg_precision = precision_score(y_resampled, y_pred_log_reg)
log_reg_recall = recall_score(y_resampled, y_pred_log_reg)
log_reg_conf_matrix = confusion_matrix(y_resampled, y_pred_log_reg)

print("Logistic Regression:")
print("Accuracy:", log_reg_accuracy)
print("Precision:", log_reg_precision)
print("Recall:", log_reg_recall)
print("Confusion Matrix:\n", log_reg_conf_matrix)

# Calculate and print metrics for Decision Tree
decision_tree_accuracy = accuracy_score(y_resampled, y_pred_decision_tree)
decision_tree_precision = precision_score(y_resampled, y_pred_decision_tree)
decision_tree_recall = recall_score(y_resampled, y_pred_decision_tree)
decision_tree_conf_matrix = confusion_matrix(y_resampled, y_pred_decision_tree)

print("\nDecision Tree:")
print("Accuracy:", decision_tree_accuracy)
print("Precision:", decision_tree_precision)
print("Recall:", decision_tree_recall)
print("Confusion Matrix:\n", decision_tree_conf_matrix)

# Calculate and print metrics for Random Forest
random_forest_accuracy = accuracy_score(y_resampled, y_pred_random_forest)
random_forest_precision = precision_score(y_resampled, y_pred_random_forest)
random_forest_recall = recall_score(y_resampled, y_pred_random_forest)
random_forest_conf_matrix = confusion_matrix(y_resampled, y_pred_random_forest)

print("\nRandom Forest:")
print("Accuracy:", random_forest_accuracy)
print("Precision:", random_forest_precision)
print("Recall:", random_forest_recall)
print("Confusion Matrix:\n", random_forest_conf_matrix)

# Calculate and print metrics for Gradient Boosting
gradient_boosting_accuracy = accuracy_score(y_resampled, y_pred_gradient_boosting)
gradient_boosting_precision = precision_score(y_resampled, y_pred_gradient_boosting)
gradient_boosting_recall = recall_score(y_resampled, y_pred_gradient_boosting)
gradient_boosting_conf_matrix = confusion_matrix(y_resampled, y_pred_gradient_boosting)

print("\nGradient Boosting:")
print("Accuracy:", gradient_boosting_accuracy)
print("Precision:", gradient_boosting_precision)
print("Recall:", gradient_boosting_recall)
print("Confusion Matrix:\n", gradient_boosting_conf_matrix)



Logistic Regression:
Accuracy: 0.6194923295903844
Precision: 0.6334634611647912
Recall: 0.5671516685117823
Confusion Matrix:
 [[42480 20750]
 [27369 35861]]

Decision Tree:
Accuracy: 0.8765933892139807
Precision: 0.8704994554224366
Recall: 0.8848173335442037
Confusion Matrix:
 [[54907  8323]
 [ 7283 55947]]

Random Forest:
Accuracy: 0.9422267910801835
Precision: 0.9949201741654572
Recall: 0.8889925668195476
Confusion Matrix:
 [[62943   287]
 [ 7019 56211]]

Gradient Boosting:
Accuracy: 0.9127866519057409
Precision: 0.9869405421540642
Recall: 0.8366439981021667
Confusion Matrix:
 [[62530   700]
 [10329 52901]]


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid for each model with a narrower range or fewer options
log_reg_params = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'C': [0.1, 1, 10]
}

decision_tree_params = {
    'max_depth': [10, 50],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

random_forest_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 50],
    'min_samples_split': [2, 5]
}

gradient_boosting_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}

# Perform randomized search for each model
log_reg_random = RandomizedSearchCV(LogisticRegression(), log_reg_params, cv=5, scoring='accuracy', n_iter=10, random_state=42)
log_reg_random.fit(X_resampled, y_resampled)
print(f"Best Logistic Regression Parameters: {log_reg_random.best_params_}")

decision_tree_random = RandomizedSearchCV(DecisionTreeClassifier(), decision_tree_params, cv=5, scoring='accuracy', n_iter=10, random_state=42)
decision_tree_random.fit(X_resampled, y_resampled)
print(f"Best Decision Tree Parameters: {decision_tree_random.best_params_}")

random_forest_random = RandomizedSearchCV(RandomForestClassifier(), random_forest_params, cv=5, scoring='accuracy', n_iter=10, random_state=42)
random_forest_random.fit(X_resampled, y_resampled)
print(f"Best Random Forest Parameters: {random_forest_random.best_params_}")

gradient_boosting_random = RandomizedSearchCV(GradientBoostingClassifier(), gradient_boosting_params, cv=5, scoring='accuracy', n_iter=10, random_state=42)
gradient_boosting_random.fit(X_resampled, y_resampled)
print(f"Best Gradient Boosting Parameters: {gradient_boosting_random.best_params_}")



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown i

Best Logistic Regression Parameters: {'solver': 'liblinear', 'C': 1}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Decision Tree Parameters: {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 50}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Random Forest Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 50}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Gradient Boosting Parameters: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1}


In [22]:
# Convert to DataFrame for feature names consistency
X_resampled_df = pd.DataFrame(X_resampled, columns=X_train.columns)
X_test_df = pd.DataFrame(X_test_scaled, columns=X_train.columns)
# Print the first few rows of the DataFrames
print("First few rows of X_resampled_df:")
print(X_resampled_df.head())

print("\nFirst few rows of X_test_df:")
print(X_test_df.head())

# Print the column names of the DataFrames
print("\nColumn names in X_resampled_df:")
print(X_resampled_df.columns)

print("\nColumn names in X_test_df:")
print(X_test_df.columns)

# Check the shapes of the DataFrames
print("\nShape of X_resampled_df:", X_resampled_df.shape)
print("Shape of X_test_df:", X_test_df.shape)

# Ensure that column names match
print("\nDo column names match between X_resampled_df and X_train?")
print(X_resampled_df.columns.equals(X_train.columns))

print("\nDo column names match between X_test_df and X_train?")
print(X_test_df.columns.equals(X_train.columns))


First few rows of X_resampled_df:
   time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0          1.874229            0.096709       -0.784130        -0.246297   
1         -0.133102           -0.207070       -0.784130        -0.615887   
2         -1.136767           -0.308330        0.390684        -1.108675   
3          1.205119            0.096709       -0.784130         0.985671   
4          2.543340            0.400488       -0.784130        -0.492690   

   number_outpatient  number_emergency  number_inpatient    diag_1    diag_2  \
0          -0.290147         -0.221346         -0.501837 -0.233059  0.335502   
1          -0.290147         -0.221346         -0.501837  2.398393  0.700212   
2          -0.290147         -0.221346         -0.501837  2.108090  1.900055   
3          -0.290147          0.904444          2.650621 -0.265835  0.906352   
4          -0.290147         -0.221346         -0.501837 -0.265835  0.340788   

     diag_3  ...  admission_

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Re-train Logistic Regression
log_reg_best = LogisticRegression(solver='liblinear', C=1)
log_reg_best.fit(X_resampled_df, y_resampled)
y_pred_log_reg_best = log_reg_best.predict(X_test_df)
print("Logistic Regression (Best Params) Classification Report:")
print(classification_report(y_test, y_pred_log_reg_best))
print("Logistic Regression (Best Params) Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_log_reg_best))

# Re-train Decision Tree
decision_tree_best = DecisionTreeClassifier(min_samples_split=5, min_samples_leaf=2, max_depth=50)
decision_tree_best.fit(X_resampled_df, y_resampled)
y_pred_decision_tree_best = decision_tree_best.predict(X_test_df)
print("\nDecision Tree (Best Params) Classification Report:")
print(classification_report(y_test, y_pred_decision_tree_best))
print("Decision Tree (Best Params) Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_decision_tree_best))

# Re-train Random Forest
random_forest_best = RandomForestClassifier(n_estimators=200, min_samples_split=2, max_depth=50)
random_forest_best.fit(X_resampled_df, y_resampled)
y_pred_random_forest_best = random_forest_best.predict(X_test_df)
print("\nRandom Forest (Best Params) Classification Report:")
print(classification_report(y_test, y_pred_random_forest_best))
print("Random Forest (Best Params) Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_random_forest_best))

# Re-train Gradient Boosting
gradient_boosting_best = GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1)
gradient_boosting_best.fit(X_resampled_df, y_resampled)
y_pred_gradient_boosting_best = gradient_boosting_best.predict(X_test_df)
print("\nGradient Boosting (Best Params) Classification Report:")
print(classification_report(y_test, y_pred_gradient_boosting_best))
print("Gradient Boosting (Best Params) Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_gradient_boosting_best))


Logistic Regression (Best Params) Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.66      0.77     27105
           1       0.17      0.56      0.27      3425

    accuracy                           0.65     30530
   macro avg       0.55      0.61      0.52     30530
weighted avg       0.84      0.65      0.71     30530

Logistic Regression (Best Params) Confusion Matrix:
[[17961  9144]
 [ 1502  1923]]

Decision Tree (Best Params) Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90     27105
           1       0.16      0.14      0.15      3425

    accuracy                           0.82     30530
   macro avg       0.53      0.52      0.52     30530
weighted avg       0.81      0.82      0.82     30530

Decision Tree (Best Params) Confusion Matrix:
[[24553  2552]
 [ 2944   481]]

Random Forest (Best Params) Classification Report:
              precision  

In [24]:
from sklearn.feature_selection import RFE

# Initialize the model for RFE
rfe_model = LogisticRegression(max_iter=5000, solver='liblinear', class_weight='balanced')

# Initialize RFE
rfe = RFE(estimator=rfe_model, n_features_to_select=10)

# Fit RFE
rfe.fit(X_resampled_df, y_resampled)

# Get the ranking of features
feature_ranking = rfe.ranking_
selected_features = X_train.columns[rfe.support_]

print("Selected Features by RFE:", selected_features)
print("Feature Ranking:", feature_ranking)


Selected Features by RFE: Index(['number_inpatient', 'number_diagnoses', 'age_[70-80)', 'age_[80-90)',
       'admission_type_id_7', 'discharge_disposition_id_11',
       'discharge_disposition_id_12', 'discharge_disposition_id_16',
       'discharge_disposition_id_17', 'discharge_disposition_id_22'],
      dtype='object')
Feature Ranking: [56  9 18 39 32 49  1 15 41 64  1 59 55 79 63 60 47 30  7  4  6  3  2  1
  1  5 73 45 54 12  8 13 62 61 44 20 21 22 31 10 27 28 76 40 65  1 71 23
  1  1 11 29 37  1  1 19 17 24 42  1 38 74 68 34 25 16 58 14 48 69 57 46
 26 33 43 36 51 78 75 66 35 77 53 50 72 52 70 67]


In [28]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

# Initialize RandomForestClassifier
random_forest = RandomForestClassifier()

# Fit Random Forest
random_forest.fit(X_resampled_df, y_resampled)

# Get feature importances
importances = random_forest.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_resampled_df.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X_resampled_df.columns[indices[f]], importances[indices[f]]))

# Select top features
top_features = X_resampled_df.columns[indices[:10]]  # Select top 10 features
print("Top Features:", top_features)



Feature ranking:
1. feature number_inpatient (0.135676)
2. feature num_procedures (0.076402)
3. feature time_in_hospital (0.074801)
4. feature diag_3 (0.070299)
5. feature number_diagnoses (0.059890)
6. feature diag_2 (0.058606)
7. feature diag_1 (0.055265)
8. feature num_medications (0.053535)
9. feature num_lab_procedures (0.048763)
10. feature gender_Male (0.042827)
11. feature number_outpatient (0.031861)
12. feature number_emergency (0.028954)
13. feature change_No (0.024881)
14. feature race_Caucasian (0.019020)
15. feature admission_source_id_7 (0.018122)
16. feature insulin_No (0.015137)
17. feature insulin_Steady (0.014735)
18. feature diabetesMed_Yes (0.012537)
19. feature admission_type_id_2 (0.011842)
20. feature age_[70-80) (0.011623)
21. feature age_[60-70) (0.009930)
22. feature discharge_disposition_id_3 (0.009853)
23. feature A1Cresult_No Test (0.009793)
24. feature age_[50-60) (0.008708)
25. feature discharge_disposition_id_6 (0.008624)
26. feature age_[80-90) (0.0085

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
# Select top features
top_features = X_resampled_df.columns[indices[:10]]

# Filter the dataset to include only the top features
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

# Resample the selected features
X_resampled_selected, y_resampled_selected = smote_tomek.fit_resample(X_train_selected, y_train)

# Initialize and train models with the selected features
log_reg = LogisticRegression(max_iter=5000, solver='saga', class_weight='balanced')
decision_tree = DecisionTreeClassifier(class_weight='balanced')
random_forest = RandomForestClassifier(class_weight='balanced')
gradient_boosting = GradientBoostingClassifier()

log_reg.fit(X_resampled_selected, y_resampled_selected)
decision_tree.fit(X_resampled_selected, y_resampled_selected)
random_forest.fit(X_resampled_selected, y_resampled_selected)
gradient_boosting.fit(X_resampled_selected, y_resampled_selected)

# Evaluate models
models = {
    "Logistic Regression": log_reg,
    "Decision Tree": decision_tree,
    "Random Forest": random_forest,
    "Gradient Boosting": gradient_boosting
}

for model_name, model in models.items():
    y_pred = model.predict(X_test_selected)
    print(f"\n{model_name} on Test Data:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, pos_label=1))
    print("Recall:", recall_score(y_test, y_pred, pos_label=1))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))



Logistic Regression on Test Data:
Accuracy: 0.551850638716017
Precision: 0.12893423051877578
Recall: 0.5202919708029197
Confusion Matrix:
 [[15066 12039]
 [ 1643  1782]]
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.56      0.69     27105
           1       0.13      0.52      0.21      3425

    accuracy                           0.55     30530
   macro avg       0.52      0.54      0.45     30530
weighted avg       0.81      0.55      0.63     30530


Decision Tree on Test Data:
Accuracy: 0.6825417622011136
Precision: 0.12522425547183352
Recall: 0.30569343065693433
Confusion Matrix:
 [[19791  7314]
 [ 2378  1047]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.73      0.80     27105
           1       0.13      0.31      0.18      3425

    accuracy                           0.68     30530
   macro avg       0.51      0.52      0.49     30530
weighted avg   